In [ ]:
import pandas as pd

from libs import ConformalClassifier, LGBMultiClassifier, AdaptativeConformalClassifier
from libs.utils import train_calib_valid_split

label = "Churn Category"
drop = ["City", "Zip Code", "Latitude", "Longitude", "Customer Status", "Churn Reason"]

df = pd.read_csv("data/telecom_customer_churn.csv", index_col="Customer ID")
df = df.drop(drop, axis=1)
df[label] = df[label].fillna("Stayed")
object_cols = df.select_dtypes("object").columns
df[object_cols] = df[object_cols].astype("category")


train, calib, valid = train_calib_valid_split(df)

In [ ]:
model = LGBMultiClassifier()
conformal_model = ConformalClassifier(model)
conformal_model.train(train, calib, valid, label)
conformal_model.calibrate(calib)
preds = conformal_model.predict(valid)

preds

In [ ]:
model = LGBMultiClassifier()
conformal_model = AdaptativeConformalClassifier(model)
conformal_model.train(train, calib, valid, label)
conformal_model.calibrate(calib)
preds = conformal_model.predict(valid)

preds